In [ ]:
#00、火上浇油====首先判断上升趋势还是下降趋势，在此趋势中做浇油法，如：下降趋势时，等高位9转，RSI>90，放量出现，
#即背离明显时做空试单，留下仓位，如继续拉高，过多，加空单。反之亦然。
###第一条最高定义，趋势由二部分组成，期商晨报+技术均线形态
#1、定义当前趋势分界线（寻找中，网络借鉴）日均线上方 OR 下方 看时间长短，当天判断。30-60分钟线，连续多长时间处于平均价之上之下
#2、上方顶背，下方底背，记录连续次数，分别看3\5\10min周期的收益和亏损
#3、止损区间，前N个周期的最高价，最低价形成一个区间(振幅)
#4、振幅很重要，计算平均震幅，预估今天低点高点，看机会点出现后，还有多大的更高更低空间，可用于止损计算
#5、止盈，止盈线逐步向上推，一旦回碰就止
#6、回归，短线（九转、RSI）与中线（MACD）错位，1分钟与5分钟、【15分钟线--仅用15分钟BBI和DKX判断不行】的错杀
#7、必须加入目前大趋势，【多】  or  【空】 只做大趋势方向
#8、收盘前平仓，不持仓  2：29 和 14：59 必平  


from jqdata import *
import pandas as pd
from datetime import date, datetime, timedelta
from jqdata import get_concepts
from pandas import DataFrame
from jqdata import get_concepts
from jqlib.technical_analysis import *



 

#=========初始变量设置================================
tdate='2024-05-04'
start_time='2024-05-06 9:05:00'
end_time = '2024-05-10 15:00:00'
end_time = datetime.strptime(end_time,'%Y-%m-%d %H:%M:%S')
current_time = datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S')
new_time = current_time + timedelta(minutes=1)
txt_new_time = new_time.strftime("%Y-%m-%d %H:%M:%S")

secID='AU9999.XSGE'

print ('回测时间：', txt_new_time,'=====',end_time,'\n 回测品种：====',secID)
count = 0
cc_d = 0
cc_k = 0
cc_k_num = 0

#===========建立测算日期区间============



#=========函数定义区=========已完成===========================
#DKX(security_list, check_date, M = 10, unit = '1d', include_now = False, fq_ref_date = None)
#多空线
def dkx_price(sec,time,unit):
    #time= '2024-04-22 09:07:00'
    dkx_now,MADKX1 = DKX(sec, check_date=time, M = 5, unit = unit, include_now = False, fq_ref_date = None)
    dkx_au=dkx_now[sec]
    
    return dkx_au
#====================================
#BBI(security_list, check_date, timeperiod1=3, timeperiod2=6, timeperiod3=12, timeperiod4=24, unit = '1d', include_now = True, fq_ref_date = None)
#用BBI判断是多头还是空头  牛熊线  与 DKX  多空线 再做对比，确认多空  5分钟为主，配合15分钟
#判断标准：BBI更灵敏，DKX偏慢。现价 > BBI > DKX 看多  当  现价<BBI<DKX 偏空
def BBI_price(sec,time,unit):
    #time= '2024-04-22 09:07:00'
    BBI1 = BBI(sec, check_date=time, timeperiod1=3, timeperiod2=6, timeperiod3=12, timeperiod4=24, unit = unit, include_now = False)
    close1 = get_price(sec, end_date=time, frequency='1m',fields=['close'],  fq='pre', count=1)
    BBI_AU=BBI1[sec]
    #print ('当天BBI为：===',BBI_AU)
    #print ('收盘价为:====',close1['close'][-1])  
    #print ('BBI检测时间为：===',time)
    if  close1['close'][-1] > BBI_AU :
        is_dk = 'D'
        #print ('当前为多头市场：===【做多】')
    else:
        #print ('当前为空头市场：===【做空】')
        is_dk = 'K'
    #print (is_dk)    
    return BBI_AU 
    




#判断当前时间是否交易时间
def var_is_trading(time,var):
    '''获取当前时间是否是该标的的交易时间,收盘不算交易时间
    传入:  date: datetime.datetime ,策略中逻辑时间为context.current_dt
           var:  标的代码,默认为AG的交易时间'''
    next_m = time + timedelta(minutes=1)
    
    temp = get_price(var,end_date=next_m,frequency='1m',count=1)
    if temp.index[-1] != next_m:
        return False
    else:
        return True
#===定义计算RSI
def make_rsi(security_list,check_date):

    rsi_1m = RSI(security_list, check_date, N1=6, unit = '1m', include_now = True, fq_ref_date = None)
    
    return rsi_1m

"""
for i in range(1,90):
    new_time = current_time + timedelta(minutes=i)
    txt_new_time = new_time.strftime("%Y-%m-%d %H:%M:%S")
    rsi_now1 = make_rsi(secID,txt_new_time)
    print (txt_new_time,'==rsi==',rsi_now1)
    
"""    
#==定义并计算分时九转    
def is_low4(sec,time):
    fs4_time = time - timedelta(minutes=4)
    close1_m = get_price(secID, end_date=time, frequency='1m',
                       fields=['open','close','open_interest'],  fq='pre', count=1)
    close4_m=get_price(secID, end_date=fs4_time, frequency='1m',
                       fields=['open','close','open_interest'],  fq='pre', count=1)
    m1=close1_m['close'][-1]
    m4=close4_m['close'][-1]
    #print (time,m1,'=======',fs4_time,m4)
    if m1 > m4 :
        islow = 1
    else:
        islow = -1
    
    return islow

#==============================================================
def istrade(Ttime,pre_price,zy_d,x):         #计算是否要 平仓 OR 持仓
    #取当前分钟 close 5个分钟价
    #动态止盈比例
    #zy_d=0.0008
    #空单止损比例
    zs_k=1 + zy_d
    #多单止损比例
    zs_d=1 - zy_d

    
    is_trade='C'
    new_Ttime = Ttime - timedelta(minutes=1)
    close_order5 = get_price(secID, end_date=new_Ttime, frequency='1m',
                       fields=['close','high','low'],  fq='pre', count=x)    
    close_db = get_price(secID, end_date=Ttime, frequency='1m',
                       fields=['close'],  fq='pre', count=1)
    close_tk = get_ticks('AU2406.XSGE',end_dt=new_Ttime,count=120,df=True)
    close_tk_min= close_tk['current'].min()
    close_tk_max= close_tk['current'].max()
    
    #print (close_tk['current'])
    #如果为空单，回撤0.02%（沪金）以内，持仓，超过 0.01%止损
    if df_cc_au.iloc[-1]['多空']  == '空'  and close_tk_max < close_order5['low'].min() * zs_k :
        is_trade='C'
        
    #if df_cc_au.iloc[-1]['多空']  == '空'  and close_tk_max > pre_price * 1.0001 :
        #is_trade='C'
    
    
    if df_cc_au.iloc[-1]['多空']  == '空' and (close_tk_max > close_order5['low'].min() * zs_k  or close_tk_max > pre_price * zs_k):
        is_trade='P'
        print ('最新价：======',close_order5['low'].min() * zs_k ,'开仓价：=====', pre_price * 0.99996)
    #如果为多单，回撤0.02%（沪金）以内，持仓，超过 0.1%止损        
    if df_cc_au.iloc[-1]['多空']  == '多'  and close_tk_min > close_order5['high'].max() * zs_d :
        is_trade='C'
    #if df_cc_au.iloc[-1]['多空']  == '多'  and close_tk_min < pre_price * 1.0001 :
        #is_trade='C'
    if df_cc_au.iloc[-1]['多空']  == '多' and (close_tk_min < close_order5['high'].max() * zs_d or close_tk_min < pre_price * zs_d ):
        is_trade='P'
        print ('最新价：======',close_order5['high'].max()* zs_d  ,'开仓价：=====', pre_price * 1.00004)
    txt_new_time = Ttime.strftime("%H:%M")
    #print ('======',txt_new_time)
    if txt_new_time == '14:59'  or txt_new_time == '2:29' :
        is_trade='P'
        print ('======== ***收盘平仓*** ======')
    return is_trade


#=========开始计算========================
df_cc_au = pd.DataFrame({'price':0, 'num':0, '多空':'0', 'time':0, '平仓收益':0},index=[0])
cc_au = pd.DataFrame(columns=['price', 'num', '多空', 'time', '平仓收益'],index=[0])
df_sum_t = pd.DataFrame(columns=['动态止损（盈）比例', '平仓收益和'],index=[0])
df_sum = pd.DataFrame(columns=['动态止损（盈）比例', '平仓收益和'],index=[0])
cc_k_num = 0
if new_time > end_time:
    print ('【结束时间】小于【开始时间】')
zy_d = 0.0001
    

while (new_time < end_time):
    
        new_time_pre = new_time - timedelta(minutes=1)
    #空单止损比例    
        zs_k=1 + zy_d
    #多单止损比例
        zs_d=1 - zy_d
        #txt_new_time = new_time.strftime("%H:%M")
        #print ('======',txt_new_time)
        #if txt_new_time == '14:59' or txt_new_time == '2:29' :
            #print ('收盘时间成功！--------------')
          

        if var_is_trading(new_time_pre,secID):
        
        
            is9 = is_low4(secID,new_time_pre)                     #计算大、小于 1 or -1
            bi=count
            count=count + is9
            if (is9 == 1) and (bi < 0 ):
                count = 0
            if (is9 == -1) and (bi > 0) :
                count = 0
            
    
    #if (count < -8) or (count > 8):
    #    print ('分时九转：',new_time,'====',count,'====')
    
    
            txt_new_time = new_time_pre.strftime("%Y-%m-%d %H:%M:%S")
            rsi_now1 = make_rsi(secID,txt_new_time)
            rsi_value = rsi_now1.get(secID)
            close1 = get_price(secID, end_date=new_time_pre, frequency='1m',fields=['close','open','high','low'],  fq='pre', count=1)
            close5m = get_price(secID, end_date=new_time_pre, frequency='5m',fields=['close','open','high','low'],  fq='pre', count=1)

            
            close_tk = get_ticks('AU2406.XSGE',end_dt=new_time_pre,count=1,df=True)
#=======先计算判断出【多】【空】方向，再进行建仓操作   目前版本是5分钟判断多空，可考虑加入15分钟和前一天的多空共振
#判断标准：BBI更灵敏，DKX偏慢。现价 > BBI > DKX 看多  当  现价<BBI<DKX 偏空
            if BBI_price(secID,new_time_pre,'5m') > dkx_price(secID,new_time_pre,'5m') :  #没加入现价 close1['close'].min() 
                is_dk = '多'
            
            else:
                
                is_dk = '空'
            
            #close_tk['current']
            if cc_k_num == 1 :
                
                if  df_cc_au.iloc[-1]['num'] == 1 :
#有持仓时，开始计算是否持仓 OR 平仓  new_time > df_cc_au.iloc[-1]['time'] and 
#=============++++++++++=============+++++++++++++++===================================
                    #close_tk = get_ticks('AU2404.XSGE',end_dt=new_Ttime,count=120,df=True)
                    #close_tk_min= close_tk['current'].min()
                    #close_tk_max= close_tk['current'].max()
                    ca = (new_time - df_cc_au.iloc[-1]['time']).total_seconds() / 60
                    check_trade = istrade(new_time,pre_price,zy_d,ca)
#=============++++++++++=============+++++++++++++++===================================
                    if check_trade == 'P' :
                        #ca = (new_time - df_cc_au.iloc[-1]['time']).total_seconds() / 60
                        print ('持仓时间（分钟）：===',ca,' 分钟')
                        #pre_price = cc_au.iloc[-1]['price']
                        #print (pre_price)
                        close_order5 = get_price(secID, end_date=new_time_pre, frequency='1m',
                                                 fields=['close','high','low'],  fq='pre', count=ca)     
                        if cc_au.iloc[-1]['多空'] == '多' :
                            cc_au['平仓收益'] = close_order5['high'].max() * zs_d - y_price
                            cc_au['price'] = close_order5['high'].max() * zs_d
                            print ('平仓收益（多）：===*****',close_order5['high'].max() * zs_d - y_price,'==******')
                            
                        else :
                            cc_au['平仓收益'] = y_price - close_order5['low'].min() * zs_k 
                            cc_au['price'] = close_order5['low'].min() * zs_k
                            print ('平仓收益(空)：===*****',y_price - close_order5['low'].min() * zs_k ,'==******')
                        
                        cc_au['num'] = 0
                        cc_au['多空'] = '平'
                        cc_au['time'] = new_time
                        cc_k_num = 0
                        df_cc_au = df_cc_au.append(cc_au, ignore_index=True)            
            if (count < -8) and rsi_value < 30   and  is_dk == '空' :   # 

                #print ('==rsi==',txt_new_time,rsi_now1)
                #print ('红九转【买点】：',new_time,'====',count,'====')
                if cc_k_num == 0 :
                    #cc_au= pd.DataFrame( {'price':close1['close'][-1], 'num': 1, '多空': '多', 'time': new_time},index=[0] )
                    
                    cc_au['price'] = close1['close'][-1]
                    cc_au['num'] = 1
                    cc_au['多空'] = '空'    #与is_dk 相反为追击反转模式   相同时为【错杀】模式
                    cc_au['time'] = new_time
                    cc_au['平仓收益'] =  0
                    cc_k_num = 1
                    pre_price = 0
                    pre_price = close1['close'][-1]  #计算最新价，初始时 = 成本价
                    y_price = 0
                    y_price = close1['close'][-1]
                    
                    print ('开仓当前状态为：【',is_dk,'】== \n',cc_au)
                    
                    df_cc_au = df_cc_au.append(cc_au, ignore_index=True)
                    #print (df_cc_au)
                    
            #rsi > 70 同时分时红九+出现，开空单 
            if rsi_value > 70 and (count > 8) and is_dk == '多' :  #  
                #print ('绿九转【卖点】：',new_time,'====',count,'====')
                if cc_k_num == 0 :
                    #cc_au= pd.DataFrame({'price':close1['close'][-1], 'num':1, '多空':'空', 'time':new_time},index=[0])
                    cc_au['price'] = close1['close'][-1]
                    cc_au['num'] = 1
                    cc_au['多空'] = '多'   #与is_dk 相反为追击反转模式   相同时为【错杀】模式
                    cc_au['time'] = new_time
                    cc_au['平仓收益'] =  0
                    cc_k_num = 1
                    pre_price = 0
                    y_price = 0
                    pre_price = close1['close'][-1]  #计算最新价，初始时 = 成本价
                    y_price = close1['close'][-1]
                    
                    print ('开仓当前状态为：【',is_dk,'】== \n',cc_au)
                    
                    df_cc_au = df_cc_au.append(cc_au, ignore_index=True)
                    #print ('取0时间：',df_cc_au['time'][0])
                    #print ('取-1时间：',df_cc_au.iloc[-1]['time'])

                        
                    #if check_trade == 'C' : 
                    #    Dt_pre_price = close1['close'][-1]
        txt_new_time = new_time.strftime("%H:%M")                
        if txt_new_time == '15:00'  :
            #print ('收盘时间成功！--------------') 
            new_time = new_time + timedelta(minutes=480)
        elif txt_new_time == '02:30':
            new_time = new_time + timedelta(minutes=390)
        else:   
            #new_time = new_time + timedelta(seconds=1)
            new_time = new_time + timedelta(minutes=1)

        if end_time == new_time + timedelta(minutes=1) :
            df_sum_t['动态止损（盈）比例'] = zy_d
            df_sum_t['平仓收益和'] = df_cc_au['平仓收益'].sum()
            df_sum = df_sum.append(df_sum_t, ignore_index=True)
            df_sum .to_excel('AU9999回测5mbbi+dkx正向多空收益汇总0415-0506-0.01-005.xlsx',encoding='utf-8')
            zy_d = zy_d + 0.0001
            new_time = current_time
            print ('当前止损比例：=============================',zy_d)
            print ('当前计算时间：=============================',new_time)

        
        if zy_d > 0.0001 :
            new_time = end_time
                
                
                
#==========================期货指标定义区+++++++++++++++++
"""
def get_q_macd5(security_list,time):
    #定义MACD取数，返回待定 周期均为5m
    ma_day=VMACD(security_list, check_date, SHORT = 12, LONG = 26, MID = 9, unit = '5m', include_now = True, fq_ref_date = None)
    
        
    return ma_day

matest=get_q_macd5(secID,time)

"""

#=========================期货指标定义区结束==========================
print (df_cc_au)
print ('计算完成！============\n',df_sum )

回测时间： 2024-05-06 09:06:00 ===== 2024-05-10 15:00:00 
 回测品种：==== AU9999.XSGE
